<a href="https://colab.research.google.com/github/Patric-Ramz/bdt-2023-26720051/blob/main/Post_Block_Assignment_1/BDT_PBA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install --quiet apache-beam[interactive]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

users_v = pd.read_csv('/content/drive/My Drive/Datasets/users_v.csv')
orders_v = pd.read_csv('/content/drive/My Drive/Datasets/orders_v.csv')

Mounted at /content/drive


In [ ]:
# Create a directory for data
!mkdir -p data

# Download kinglear.txt from Cloud Storage
!gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt data/

# Display the number of lines in kinglear.txt
!wc -l data/kinglear.txt

# Display the first three lines of kinglear.txt
!head -3 data/kinglear.txt

# Import necessary Python packages
import apache_beam as beam
import re

# Set input and output patterns
inputs_pattern = 'data/*'
outputs_prefix = 'outputs/part'

Copying gs://dataflow-samples/shakespeare/kinglear.txt...
/ [1 files][153.6 KiB/153.6 KiB]                                                
Operation completed over 1 objects/153.6 KiB.                                    
5525 data/kinglear.txt
	KING LEAR




In [ ]:
import apache_beam as beam
from datetime import datetime

def format_user(element):
    # Split each line based on comma
    fields = element.split(',')

    user_id = fields[0]
    full_name = fields[1].split()
    first_name = full_name[0]
    last_name = ' '.join(full_name[1:])

    gender = "Male" if fields[2] == "male" else "Female"

    # Categorizing the age
    age = int(fields[3])
    age_category = "16-55" if 16 <= age <= 55 else "56+"

    # Splitting address into City, State and Zip code
    address_parts = fields[4].rsplit('-', 2)
    city = address_parts[0]
    state = address_parts[1]
    zip_code = address_parts[2]

    # Changing date format
    date_format = "%Y/%m/%d"
    date_joined = datetime.strptime(fields[5], date_format).strftime('%Y-%m-%d')

    return f'{user_id}; {first_name} {last_name}; {gender}; {age_category}; {city}, {state}, {zip_code}; {date_joined}'

with beam.Pipeline() as p:
    (
        p | 'Read CSV File' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/users_v.csv', skip_header_lines=1)
          | 'Format Users' >> beam.Map(format_user)
          | 'Write to Output CSV' >> beam.io.WriteToText('/content/drive/My Drive/Datasets/marketing_format', header='User Id;Name Surname;Male/Female;16-55;City,state,zip code;YYYY-MM-dd', file_name_suffix='.csv', shard_name_template='')
    )

In [ ]:
# Question 1

import apache_beam as beam

# Define a function to extract the user_id from users.csv
def extract_user_key(element):
    fields = element.split(',')
    user_id = fields[0]
    return (user_id, element)

# Define a function to extract the user_id from orders.csv
def extract_order_key(element):
    fields = element.split(',')
    user_id = fields[1]  # Assuming user_id is the second field in orders.csv
    return (user_id, element)

# Define a function to format the combined output
def format_output(element):
    user_id, collections = element
    user_info = collections['users']
    orders = collections['orders']

    # Return a combined result for further processing or writing to an output
    return (user_id, user_info, orders)

with beam.Pipeline() as p:
    users = (
        p | 'Read Users' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/users_v.csv', skip_header_lines=1)
          | 'Extract User Key' >> beam.Map(extract_user_key)
    )

    orders = (
        p | 'Read Orders' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/orders_v.csv', skip_header_lines=1)
          | 'Extract Order Key' >> beam.Map(extract_order_key)
    )

    results = (
        {'users': users, 'orders': orders}
        | 'CoGroupByKey' >> beam.CoGroupByKey()
        | 'Format Output' >> beam.Map(format_output)
        # You can add additional transforms here, or write to an output
    )

In [ ]:
# Question 2

import apache_beam as beam

def extract_gender_and_order(element):
    user_id, collections = element
    user_info = collections['users'][0]  # Assuming one user record per ID
    orders = collections['orders']

    # Extract gender from user info
    fields = user_info.split(',')
    gender = fields[2]  # Assuming gender is the third field in users_v.csv

    return (gender, len(orders))

def compute_average(element):
    gender, order_counts = element
    total_orders = sum(order_counts)
    number_of_customers = len(order_counts)
    average_orders = total_orders / number_of_customers
    return (gender, average_orders)

with beam.Pipeline() as p:
    users = (
        p | 'Read Users' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/users_v.csv', skip_header_lines=1)
          | 'Extract User Key' >> beam.Map(extract_user_key)
    )

    orders = (
        p | 'Read Orders' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/orders_v.csv', skip_header_lines=1)
          | 'Extract Order Key' >> beam.Map(extract_order_key)
    )

    average_orders_by_gender = (
        {'users': users, 'orders': orders}
        | 'CoGroupByKey' >> beam.CoGroupByKey()
        | 'Extract Gender and Order Count' >> beam.Map(extract_gender_and_order)
        | 'Group By Gender' >> beam.GroupByKey()
        | 'Compute Average by Gender' >> beam.Map(compute_average)
        | 'Print Results' >> beam.Map(print)
    )

('female', 342.1982608695652)
('male', 341.89312344656173)


In [ ]:
# Question 3

import apache_beam as beam
from apache_beam import pvalue

class SplitOutput(beam.DoFn):
    def process(self, element, orders_total):
        gender, order_counts = element
        total_orders = sum(order_counts)
        number_of_customers = len(order_counts)
        average_orders = total_orders / number_of_customers

        # Update the total number of orders processed
        orders_total.inc(total_orders)

        return [(gender, average_orders)]

with beam.Pipeline() as p:
    users = (
        p | 'Read Users' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/users_v.csv', skip_header_lines=1)
          | 'Extract User Key for Users' >> beam.Map(extract_user_key)
    )

    orders = (
        p | 'Read Orders' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/orders_v.csv', skip_header_lines=1)
          | 'Extract Order Key for Orders' >> beam.Map(extract_order_key)
    )

    orders_total = beam.metrics.Metrics.counter('order', 'total')

    average_orders_by_gender = (
        {'users': users, 'orders': orders}
        | 'CoGroupByKey' >> beam.CoGroupByKey()
        | 'Extract Gender and Order Count' >> beam.Map(extract_gender_and_order)
        | 'Group By Gender' >> beam.GroupByKey()
        | 'Compute Average by Gender' >> beam.ParDo(SplitOutput(), orders_total=orders_total)
        | 'Print Average Results' >> beam.Map(print)
    )

    # Printing the total orders after the pipeline finishes
    result = p.run()
    result.wait_until_finish()
metrics = result.metrics().query()
for metric in metrics['counters']:
    if metric.key.metric.name == 'total':
        print(f"Total number of orders processed: {metric.committed}")

('male', 341.89312344656173)
('female', 342.1982608695652)
('female', 342.1982608695652)
('male', 341.89312344656173)
Total number of orders processed: 806193


In [ ]:
# Question 5

import apache_beam as beam

# Step 1: Define a function to categorize users into age groups.
def categorize_by_age_group(element):
    user_id, collections = element
    user_info = collections['users'][0]  # Assuming one user record per ID

    # Extract age from user info
    fields = user_info.split(',')
    age = int(fields[3])  # Assuming age is the fourth field in users_v.csv

    # Categorize into age groups
    if 16 <= age < 26:
        age_group = "16-26"
    elif 26 <= age < 36:
        age_group = "26-36"
    elif 36 <= age < 46:
        age_group = "36-46"
    elif 46 <= age < 56:
        age_group = "46-56"
    else:
        age_group = "Other"

    return (age_group, len(collections['orders']))

# Step 2: Read the users and orders datasets.
with beam.Pipeline() as p:
    users = (
        p | 'Read Users' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/users_v.csv', skip_header_lines=1)
          | 'Extract User Key for Users' >> beam.Map(extract_user_key)
    )

    orders = (
        p | 'Read Orders' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/orders_v.csv', skip_header_lines=1)
          | 'Extract Order Key for Orders' >> beam.Map(extract_order_key)
    )

    # Step 3: Join these datasets based on user_id.
    # Step 4: Extract and group by age categories.
    age_group_orders = (
        {'users': users, 'orders': orders}
        | 'CoGroupByKey' >> beam.CoGroupByKey()
        | 'Categorize by Age Group' >> beam.Map(categorize_by_age_group)
        | 'Count Orders by Age Group' >> beam.CombinePerKey(sum)
        | 'Print Results' >> beam.Map(print)
    )

('Other', 323193)
('26-36', 137232)
('16-26', 96019)
('36-46', 132048)
('46-56', 117701)


In [ ]:
# Question 6

import apache_beam as beam

# Step 1: Filter out orders where spinach was purchased.
def filter_spinach_orders(element):
    fields = element.split(',')
    product_name = fields[2]  # Assuming product name is the third field in orders_v.csv
    return product_name.lower() == "spinach"

# Step 2: Categorize users into age groups and count spinach purchases.
def categorize_by_age_group_and_count(element):
    user_id, collections = element
    user_info = collections['users'][0]  # Assuming one user record per ID

    # Extract age from user info
    fields = user_info.split(',')
    age = int(fields[3])  # Assuming age is the fourth field in users_v.csv

    # Categorize into age groups
    if 16 <= age < 26:
        age_group = "16-26"
    elif 26 <= age < 36:
        age_group = "26-36"
    elif 36 <= age < 46:
        age_group = "36-46"
    elif 46 <= age < 56:
        age_group = "46-56"
    else:
        age_group = "Other"

    spinach_purchases = len(collections['orders'])

    return (age_group, spinach_purchases)

with beam.Pipeline() as p:
    users = (
        p | 'Read Users' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/users_v.csv', skip_header_lines=1)
          | 'Extract User Key for Users' >> beam.Map(extract_user_key)
    )

    spinach_orders = (
        p | 'Read Orders' >> beam.io.ReadFromText('/content/drive/My Drive/Datasets/orders_v.csv', skip_header_lines=1)
          | 'Filter Spinach Orders' >> beam.Filter(filter_spinach_orders)
          | 'Extract Order Key for Spinach Orders' >> beam.Map(extract_order_key)
    )

    age_group_spinach_counts = (
        {'users': users, 'orders': spinach_orders}
        | 'CoGroupByKey' >> beam.CoGroupByKey()
        | 'Categorize by Age Group and Count Spinach' >> beam.Map(categorize_by_age_group_and_count)
        | 'Count Spinach Purchases by Age Group' >> beam.CombinePerKey(sum)
        | 'Print Results' >> beam.Map(print)
    )

('Other', 1773)
('26-36', 724)
('16-26', 495)
('36-46', 664)
('46-56', 586)
